In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import os.path as osp
import matplotlib.pyplot as plt

import torch

from src.datasets import DatasetBuilder
from src.utils import seed_everything, get_config, load_model_weights, evaluate_classification_model
from src.cf_methods.coin import CounterfactualCGAN

seed_everything()

2025-05-14 22:50:51.371117: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747255851.389042  548550 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747255851.394524  548550 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747255851.409255  548550 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747255851.409272  548550 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747255851.409274  548550 computation_placer.cc:177] computation placer alr

In [7]:
config_dir = '/data/leuven/365/vsc36567/CF-Robustness-Benchmark/configs'
config_path = osp.join(config_dir, 'coin_derma.yaml')
config = get_config(config_path)

In [8]:
cfcgan = CounterfactualCGAN(opt=config, img_size=config.data.img_size)

Using SNConv in generator: False
Generator in channels [1024, 1536, 768, 384, 192]
Generator out channels [1024, 512, 256, 128, 64]
Using SNConv in generator: False
Using perturbation fuse scheme: skip_add_tanh
